# 进展记录
该脚本主要记录实验进展，包括但不限于实验设置、实验结果、分析、疑惑记录等内容

# 11月

## 11.26
### 记录
`logs/rl_games/leaphand_object_rot/2025-11-26_10-28-35` 关节空间
<br> `logs/rl_games/leaphand_se3_rot/2025-11-26_10-03-32` se3末端刚体旋量空间
<br> `logs/rl_games/leaphand_se3_rot/2025-11-26_21-48-16` se3缩放修正版
#### 分析与疑惑
- 试验了se3动作空间的训练效果，初步结果不太理想。主要问题在于奖励收敛很慢，且最终效果不如关节空间动作。怀疑原因有：1是se3动作空间的高维度和冗余性导致训练难度加大；2是当前的奖励设计和观察空间并未针对se3动作进行优化，导致学习效率低下。
- 先做了简单的对比训练，除了动作项外其他都一致。se3动作能做，但奖励收敛很慢。同时发现了一些奇怪的现象：1是se3手指和物体不怎么接触，很多是靠指腹带动物体转；2是实际关节动作抖动性不算太大，但经常一阵一阵的反复，不知道是se(3)旋量和J_b的伪逆导致的这种不稳定行为不。
- 传统的EMA，是在神经网络外部显示裁减增量幅度来达到动作平滑化，并非网络内生平滑。但我目前如果在深度学习这块去做，不知道来不来得及，但实现神经网络的物理内生一致性，达到对于大多数任务所需要的平滑一致动作输出，确实是个潜在的idea
- 虽然初次训练效果一般，但我仍坚信se(3)的应用潜力。也许每个动作表征空间都有自身适用的机制，它们的偏好分布具有偏差，强制相同条件下去比对也未必公平。或许当下阶段，不是在当前关节空间最合适的配置里去调se3旋量动作空间的效果，而是从se3本身出发，去调整适配于se3自身的观察、奖励等，以达到效果更好。真正的比较应该是SOTA比SOTA，关节空间SOTA比se3旋量空间SOTA
- 非常奇怪，修改了se3actions.py里的缩放，但训练效果反而更差了，不如之前的版本。怀疑是哪里有bug，待排查
> 更正：缩放修正后后期奖励比未修正的要好，最后阶段的奖励是未修正时的4倍左右，可能只是前面慢一点；
### 计划
开始做针对se3的适配工作，分为5大块
1. 设置适合se(3)的观察空间。
<br>感觉关节量可能不是很合适，因为这相当于仍要学习一个正向运动学的过程构建，网络会隐式地包含手的结构信息。但本体感知的信息本身就很少，关节量适合用作基准参照。
难道说，这时的本体感知变成了目标旋量动作？
理想中更合适的相对信息，来自视触觉模态信息。一种是点云，手和物体点云所构成的相对信息。一种是触觉，如视触觉。
重点可探索点云方面的视觉信息，最好是即时可用的。
IDEA：不是说结构信息不必学，是想让策略网络更关注任务本身的层面。结构这块的信息，或许可以参照GET-Zero训练一个上游特征的结构提取基座大模型G: q -->q* ，但怎么去理解手的结构本身就是一个难题
1. 奖励函数设计的调整 (Crucial for se3)
<br>关于动作稳定和平滑，可以改为惩罚SE(3)能量 ${\mathcal{V}_b^{T}}M\mathcal{V}_b$ ，以及 。
关于操作多样性，可用可操作度指标 (Manipulability Index) $det(J{{J}^{T}})$ 或 雅可比条件数 $\frac{{{\sigma }_{max}}(J)}{{{\sigma }_{min}}(J)}$ 。
<br>另外可以考虑触觉方面的奖励，这个可能要调用IsaacLab的触觉传感器
1. 动作的微调
<br>先想的是额外再添加一个动作缩放因子  
可选：自适应 $\lambda$，具体实现方式待定；PD控制中引入位置误差补偿（Closed-Loop Inverse Kinematics, CLIK），有无必要确认：
1. 网络的设置
<br>现有的动作维度为24，以及考虑到如果用关节作为本体感知信息，那终究还是要隐式的学习手的动力学结构，而手指为4个节点的树，那是否意味着用4层MLP会更好？还有动作空间匹配的问题
1. 学习算法
<br>不知道rl_games最新的pbt训练是否对训练效果的提升或者超参数的调整有所增益？SAC算法和TD3算法和PPO相比又如何

## 11.27
### 计划与行动
今天主要针对奖励这块修正。昨天的实验，根据奖励管理器的记录，我觉得pose_diff权重偏高，而且其设置的默认位形可能比较奇异，我感觉是不太好的。关于pose_diff，我可以重新人为设置和调整下更合适的初始位形，甚至有一个
IDEA：利用别人预训练好的抓取网络，生成一个抓取姿态反解得到初始位形（这样可以保证手指和物体有良好的接触、且可针对不同物体进行泛化）。后期训练过程中可以通过课程学习逐渐降低pose_diff的权重。

1. 奖励的设计与调整
<br>主要想调整pose_diff和操作度指标这两块，思路和昨天设想的一致。
<br>
2. 可操作度分析
<br> `source/leaphand/leaphand/tasks/functional/test_manipulability.py`。对操作度、雅可比条件数的计算和显示进行了修正，添加了打印按钮，可以将当前各手指的雅可比矩阵和奇异值信息打印到终端，便于分析。
- 从结果来看，条件数普遍较大（>50)，操作度都很小(<0.1)，这可能是每根手指自由度数量较少的缘故（4个自由度），导致雅可比矩阵接近奇异矩阵。
- LeapHand 是一个 4-DOF 手指系统，需要在 6D 空间（SE(3)）操作。这是一个欠驱动系统 (m=6 > n=4)。即使在最佳位形，手指也无法实现任意 6D 运动（只能实现 4D 子空间的运动）。高条件数和低可操作度是欠驱动系统的固有特性，不一定是错误
- 所以，该如何设计奖励呢？这是一个问题啊，操作度和条件数变化幅度较大。难道也是从课程学习的角度，一开始奖励操作度较高的、鼓励探索，后面降低权重？因为完成任务本身和位形对应的操作度可能并无直接关联
- IDEA：从任务角度来说，我觉得末端的三位位置相对更重要些，同时因为4-Dof，导致位置和姿态高度耦合，所以学习的时候，如果能学到比较好的位置、姿态一样也会比较好。这一过程该如何数学建模呢，只关心旋量的平移部分，让学习的维度降下来？
- NOTE：如果想反映位置的重要性，可用位置可操作度，$J_{\text{pos}} \in \mathbb{R}^{3 \times 4}$，即只取雅可比矩阵的前三行（Physx里前三行是线速度）来计算可操作度和条件数，或许能更好地反映位置控制能力；或者适用加权雅可比矩阵，通过单位换算和权重调整，人为地“缩小”姿态空间的重要性，从而让奇异值分解（SVD）主要反映位置能力，同时兼顾姿态。既保持位置灵活性，又防止整体奇异。

## 11.28
### 计划
1. 先回顾一下 `Velocity Kinematics and statics`
2. 继续调整奖励函数，并争取当晚能继续做一组对比实验
### 问题
1. 是否可以从urdf或usd文件获取手指的解析雅可比矩阵？如果能够获取，感觉可以加速计算，或者让我对手的操作性有个更直观的认知。
> 获取了感觉也没太大用，计算实际用不到，而且误差较大。
> 但是我们可以通过urdf的解析雅可比矩阵，做可操作性的理论分析，给我们直观认知。特别是手指末端的位置雅可比及操作度
2. NOTE：除了操作度椭球，还有力矩椭球。两者在相同位形上是正好成反比的，操作度椭球在某个方向上越大，力矩椭球在该方向上就越小。该如何衡量呢？


## 12.2
### 计划
1. 不用想太多，明确操作度或条件数的用途，添加相应的奖励
2. 新思路，加权雅可比矩阵


### 加权雅可比矩阵 (Weighted Jacobian)

#### 问题背景
DLS 伪逆基于完整的 6×4 雅可比矩阵，但位置和姿态的**单位不一致**（米 vs 弧度）且**重要性可能不同**。加权雅可比通过引入权重矩阵来解决这个问题。

#### 数学形式

**任务空间加权（左乘）**
$$J_w = W_x \cdot J$$
其中 $W_x \in \mathbb{R}^{6\times6}$ 为任务空间权重矩阵：
$$W_x = \begin{bmatrix} W_\omega & 0 \\ 0 & W_v \end{bmatrix} = \begin{bmatrix} w_\omega I_3 & 0 \\ 0 & w_v I_3 \end{bmatrix}$$

**关节空间加权（右乘）**
$$J_w = J \cdot W_q^{-1}$$
其中 $W_q \in \mathbb{R}^{4\times4}$ 为关节空间权重矩阵，用于约束关节运动。

**完整加权形式**
$$J_w = W_x \cdot J \cdot W_q^{-1}$$

#### 加权 DLS 伪逆

加权最小二乘问题：
$$\min_{\dot\theta} \| W_x(J\dot\theta - \mathcal{V}) \|^2 + \lambda^2 \| W_q \dot\theta \|^2$$

对应的加权 DLS 解：
$$\dot\theta = W_q^{-1} J_w^T (J_w J_w^T + \lambda^2 I)^{-1} W_x \mathcal{V}$$

其中 $J_w = W_x J W_q^{-1}$。

#### 单位归一化策略

**问题：** PhysX 雅可比中，角速度 $\omega$ (rad/s) 和线速度 $v$ (m/s) 量级差异大
- 角速度：$\|\omega\| \sim O(1)$
- 线速度：$\|v\| \sim O(10^{-2})$（手指尺度约 5cm）

**归一化权重设计：**
$$w_v = \frac{1}{L_{char}}, \quad w_\omega = 1$$
- $L_{char}$：特征长度，如手指长度 $\approx 0.05m$
- 这样 $w_v \cdot v$ 与 $\omega$ 量级一致

归一化后的加权雅可比：
$$J_w = \begin{bmatrix} I_3 & 0 \\ 0 & \frac{1}{L_{char}} I_3 \end{bmatrix} \cdot J = \begin{bmatrix} J_\omega \\ \frac{1}{L_{char}} J_v \end{bmatrix}$$

#### 任务优先级加权

当位置控制更重要时（你在 11.27 笔记中的想法）：
$$W_x = \begin{bmatrix} w_\omega I_3 & 0 \\ 0 & w_v I_3 \end{bmatrix}, \quad w_v > w_\omega$$

**典型参数设置：**
- 位置优先：$w_v = 1.0, w_\omega = 0.1$
- 单位归一化 + 位置优先：$w_v = 20.0$ ($\frac{1}{L_{char}}$), $w_\omega = 0.5$

#### 加权操作度指标

使用加权雅可比计算操作度：
$$w_{weighted} = \sqrt{\det(J_w^T J_w)} = \sqrt{\det(W_q^{-1} J^T W_x^2 J W_q^{-1})}$$

加权条件数：
$$\kappa_w = \frac{\sigma_{max}(J_w)}{\sigma_{min}(J_w)}$$

**优势：** 
- 消除单位不一致带来的奇异值分布畸变
- 更真实地反映任务相关的操作能力
- 条件数更稳定，可作为更可靠的奖励信号